In [1]:
!nvidia-smi

Wed Aug 26 11:59:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   41C    P0    57W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import transformers

import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm import tqdm

In [3]:
tf.__version__

'2.3.0'

In [4]:
train_df = pd.read_json('data/train.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [5]:
val_df = pd.read_json('data/dev.jsonl', lines = True)
val_df["img"] = "data/" + val_df["img"]

In [6]:
img_width = 224
img_height = 224

In [7]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [8]:
max_len = 128

### Train

In [9]:
def build_model():
    
    encoder = transformers.TFDistilBertModel.from_pretrained("distilbert-base-uncased")
    encoder.trainable = False

    input_img = tf.keras.layers.Input(
        shape = (img_width, img_height, 3), name = "image"
    )
    
    input_ids = tf.keras.layers.Input(name = "input_ids", shape = (max_len,), dtype = tf.int32)

    attention_mask = tf.keras.layers.Input(name = "attention_mask", shape = (max_len,), dtype = tf.int32)

    y = encoder(input_ids, attention_mask = attention_mask)[0]
    y = tf.keras.layers.BatchNormalization()(y)
    
    y = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True))(y)

    extractor = tf.keras.applications.EfficientNetB7(include_top = False, \
                                                 input_tensor = input_img, weights = "imagenet")
    
    extractor.trainable = False
    
    for layer in extractor.layers:
        if(layer.name.startswith("block7") or layer.name.startswith("block6")):
            layer.trainable = True
    
    x = tf.keras.layers.BatchNormalization()(extractor.output)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    
    y = tf.keras.layers.BatchNormalization()(y)
    y = tf.keras.layers.GlobalMaxPooling1D()(y)
    
    x = tf.keras.layers.concatenate([x, y])
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dense(1024)(x)
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x = tf.keras.layers.Dense(128)(x)
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    out = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = tf.keras.models.Model([input_img, input_ids, attention_mask], out)

    return model

In [10]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = build_model()
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [11]:
def encode_single_sample(img_path, label, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf')
    return {"image": img, "label": label, "text": text}

In [12]:
class HatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, label, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text
        self.label = label

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_label = self.label[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        z = np.zeros((self.batch_size), dtype = "float32")
        
        for i,j,k,l in zip(range(self.batch_size), batch_input_img_paths, batch_label, batch_text):
            sample = encode_single_sample(j,k,l)
            w[i] = sample["image"].numpy().tolist()
            input_ids = sample["text"]['input_ids'].numpy().tolist()[0]
            x[i] = input_ids + [0]*(max_len - len(input_ids))
            attention_mask = sample["text"]['attention_mask'].numpy().tolist()[0]
            y[i] = attention_mask + [0]*(max_len - len(attention_mask))
            z[i] = sample["label"]
        
        return [w,x,y], z

In [13]:
train_gen = HatefulMemes(1, train_df["img"].values.tolist(), train_df["label"].values.tolist(), \
                         train_df["text"].values.tolist())
val_gen = HatefulMemes(1, val_df["img"].values.tolist(), val_df["label"].values.tolist(), \
                         val_df["text"].values.tolist())

In [14]:
x_train, y_train, x_test, y_test = [], [], [], []
for i,j in tqdm(train_gen, total = 8500):
    x_train.append(i)
    y_train.append(j)
for i,j in tqdm(val_gen, total = 500):
    x_test.append(i)
    y_test.append(j)

100%|██████████| 500/500 [00:56<00:00,  8.84it/s]


In [ ]:
epochs = 50
early_stopping_patience = 5

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_accuracy", patience = early_stopping_patience, restore_best_weights = True
)

history = model.fit(x_train, y_train, batch_size = 32, \
                    validation_data = (x_test, y_test), epochs = epochs, callbacks = [early_stopping])

### Predict

In [ ]:
train_df = pd.read_json('data/test.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [ ]:
train_df

In [ ]:
def val_encode_single_sample(img_path, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf')
    return {"image": img, "text": text}

In [ ]:
class ValHatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        
        for i,j,k in zip(range(self.batch_size), batch_input_img_paths, batch_text):
            sample = val_encode_single_sample(j,k)
            w[i] = sample["image"].numpy().tolist()
            input_ids = sample["text"]['input_ids'].numpy().tolist()[0]
            for _ in range(max_len - len(input_ids)):
              input_ids.append(0)
            x[i] = input_ids
            attention_mask = sample["text"]['attention_mask'].numpy().tolist()[0]
            for _ in range(max_len - len(attention_mask)):
              attention_mask.append(0)
            y[i] = attention_mask
        
        return [w,x,y]

In [ ]:
pred_gen = ValHatefulMemes(1, train_df["img"].values.tolist(), train_df["text"].values.tolist())

In [ ]:
preds = model.predict(pred_gen, verbose = 1)

In [ ]:
def prob2pred(x):
    if x > 0.5:
        return 1
    else:
        return 0

In [ ]:
train_df["proba"] = [i[0] for i in preds.tolist()]
train_df["label"] = train_df["proba"].apply(prob2pred)

In [ ]:
train_df = train_df.drop(["img","text"], axis = 1)
train_df

In [ ]:
train_df.to_csv("submission_new.csv", index = False)